In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import os
import shap
from lime import lime_tabular
import pickle

# 생존 분석 라이브러리
from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.statistics import logrank_test
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.metrics import concordance_index_censored

# 머신러닝 라이브러리
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import joblib

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

def setup_korean_font():
    """한글 폰트 설정"""
    import platform
    import matplotlib.font_manager as fm
    
    system = platform.system()
    
    if system == 'Windows':
        try:
            plt.rcParams['font.family'] = 'Malgun Gothic'
        except:
            try:
                font_path = 'C:/Windows/Fonts/malgun.ttf'
                font_name = fm.FontProperties(fname=font_path).get_name()
                plt.rc('font', family=font_name)
            except:
                print("⚠️ 한글 폰트 설정 실패")
    elif system == 'Darwin':
        plt.rcParams['font.family'] = 'AppleGothic'
    else:
        plt.rcParams['font.family'] = 'NanumGothic'
    
    plt.rcParams['axes.unicode_minus'] = False
    print("✅ 한글 폰트 설정 완료")

class LiverCancerTreatmentEffectPredictorCDSS:
    """간암 치료 효과 예측 모델 클래스 (CDSS 호환 + XAI + 과적합 방지)"""
    
    def __init__(self, data_path):
        setup_korean_font()
        self.data_path = data_path
        self.df = None
        self.processed_df = None
        self.models = {}
        self.results = {}
        self.feature_names = []
        self.scaler = None
        self.label_encoders = {}
        self.shap_explainers = {}
        self.shap_values = {}
        self.lime_explainers = {}
        self.holdout_patient = None
        
        print(f"🚀 간암 치료 효과 예측 모델 초기화 (CDSS 호환 + XAI + 과적합 방지)")
        print(f"📁 데이터 경로: {data_path}")
        print(f"⏰ 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*60)
    
    def load_and_explore_data(self):
        """데이터 로드 및 탐색적 분석"""
        print("\n📊 1. 데이터 로드 및 탐색")
        
        try:
            self.df = pd.read_csv(self.data_path)
            print(f"✅ 데이터 로드 성공: {self.df.shape[0]}행 × {self.df.shape[1]}열")
        except Exception as e:
            print(f"❌ 데이터 로드 실패: {e}")
            return False
        
        # 기본 정보 출력
        print(f"📈 데이터 기본 정보:")
        print(f"   - 총 환자 수: {len(self.df)}")
        print(f"   - 총 컬럼 수: {len(self.df.columns)}")
        
        # 생존 상태 분포
        if 'vital_status' in self.df.columns:
            status_counts = self.df['vital_status'].value_counts()
            print(f"   - 생존 환자: {status_counts.get('Alive', 0)}명")
            print(f"   - 사망 환자: {status_counts.get('Dead', 0)}명")
            print(f"   - 사망률: {status_counts.get('Dead', 0)/len(self.df)*100:.1f}%")
        
        return True
    
    def preprocess_data(self):
        """데이터 전처리"""
        print("\n🔧 2. 데이터 전처리")
        
        # 치료 효과 예측 모델용 선택된 컬럼들
        selected_columns = [
            # 생존 결과 변수
            'vital_status', 'days_to_death', 'days_to_last_follow_up',
            # 간기능 평가 (핵심 예측 인자)
            'child_pugh_classification', 'ishak_fibrosis_score',
            # 병기 및 종양 특성
            'ajcc_pathologic_stage', 'ajcc_pathologic_t', 'ajcc_pathologic_n', 'ajcc_pathologic_m',
            'tumor_grade', 'morphology',
            # 환자 기본 특성
            'age_at_diagnosis', 'gender', 'race', 'ethnicity',
            # 치료 관련 변수
            'treatments_pharmaceutical_treatment_intent_type', 
            'treatments_pharmaceutical_treatment_type',
            'prior_treatment',
            # 추가 임상 변수
            'primary_diagnosis', 'year_of_diagnosis'
        ]
        
        # 존재하는 컬럼만 선택
        available_columns = [col for col in selected_columns if col in self.df.columns]
        missing_columns = [col for col in selected_columns if col not in self.df.columns]
        
        print(f"✅ 사용 가능한 컬럼: {len(available_columns)}개")
        if missing_columns:
            print(f"⚠️  누락된 컬럼: {missing_columns}")
        
        self.processed_df = self.df[available_columns].copy()
        
        # 생존 시간 및 이벤트 변수 생성
        print("🔄 생존 변수 생성 중...")
        self.processed_df['event'] = (self.processed_df['vital_status'] == 'Dead').astype(int)
        
        # 생존 시간 계산
        self.processed_df['duration'] = self.processed_df['days_to_death'].fillna(
            self.processed_df['days_to_last_follow_up']
        )
        
        # 치료 효과 지표 생성
        self.processed_df['treatment_effectiveness'] = np.where(
            self.processed_df['duration'] > self.processed_df['duration'].median(), 1, 0
        )
        
        # 유효하지 않은 생존 시간 제거
        valid_mask = (self.processed_df['duration'].notna()) & (self.processed_df['duration'] > 0)
        self.processed_df = self.processed_df[valid_mask].copy()
        
        print(f"✅ 유효한 생존 데이터: {len(self.processed_df)}명")
        print(f"   - 사망 이벤트: {self.processed_df['event'].sum()}건")
        print(f"   - 중간 생존 시간: {self.processed_df['duration'].median():.0f}일")
        print(f"   - 치료 효과 양호: {self.processed_df['treatment_effectiveness'].sum()}명")
        
        # 결측값 분석
        print("\n📋 결측값 분석:")
        missing_analysis = self.processed_df.isnull().sum()
        missing_percent = (missing_analysis / len(self.processed_df) * 100).round(1)
        
        for col in missing_analysis[missing_analysis > 0].index:
            print(f"   - {col}: {missing_analysis[col]}개 ({missing_percent[col]}%)")
        
        # 높은 결측률 컬럼 제거 (80% 이상)
        high_missing_cols = missing_percent[missing_percent > 80].index.tolist()
        if high_missing_cols:
            print(f"🗑️  높은 결측률 컬럼 제거: {high_missing_cols}")
            self.processed_df = self.processed_df.drop(columns=high_missing_cols)
        
        return True
    
    def prepare_features(self):
        """특성 준비 및 인코딩 (CDSS 호환)"""
        print("\n🎯 3. 특성 준비 및 인코딩")
        
        # CDSS 테스트용 환자 1명 미리 분리
        print("🔄 CDSS 테스트용 환자 분리 중...")
        holdout_idx = self.processed_df.sample(n=1, random_state=42).index[0]
        self.holdout_patient = self.processed_df.loc[holdout_idx:holdout_idx].copy()
        remaining_df = self.processed_df.drop(holdout_idx).copy()
        
        print(f"   - CDSS 테스트 환자: {holdout_idx}")
        print(f"   - 모델 훈련용 데이터: {len(remaining_df)}명")
        
        # 특성과 타겟 분리
        feature_cols = [col for col in remaining_df.columns 
                       if col not in ['vital_status', 'days_to_death', 'days_to_last_follow_up', 
                                     'event', 'duration', 'treatment_effectiveness']]
        
        X = remaining_df[feature_cols].copy()
        y_duration = remaining_df['duration'].values
        y_event = remaining_df['event'].values.astype(bool)
        y_effectiveness = remaining_df['treatment_effectiveness'].values
        
        print(f"📊 초기 특성 개수: {len(feature_cols)}")
        print(f"📊 샘플 개수: {len(X)}")
        
        # 범주형 변수 인코딩
        categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
        numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()
        
        print(f"🔤 범주형 변수: {len(categorical_cols)}개")
        print(f"🔢 수치형 변수: {len(numerical_cols)}개")
        
        # 결측값 처리
        print("🔄 결측값 처리 중...")
        
        # 임상적으로 의미있는 Unknown 값을 가질 수 있는 컬럼들
        meaningful_unknown_cols = [
            'child_pugh_classification', 'ajcc_pathologic_stage', 'ajcc_pathologic_t',
            'ajcc_pathologic_n', 'ajcc_pathologic_m', 'tumor_grade',
            'treatments_pharmaceutical_treatment_intent_type',
            'treatments_pharmaceutical_treatment_type', 'prior_treatment'
        ]
        
        self.label_encoders = {}
        
        for col in categorical_cols:
            if col in X.columns:
                print(f"\n   🔍 {col} 처리:")
                
                # 현재 값 분포 확인
                value_counts = X[col].value_counts(dropna=False)
                print(f"      - 전처리 전 분포: {dict(list(value_counts.items())[:3])}")
                
                # 'NA' 문자열을 결측치로 변환
                if 'NA' in X[col].values:
                    X[col] = X[col].replace('NA', np.nan)
                    print(f"      - 'NA' 문자열을 결측치로 변환")
                
                # Unknown 값 처리 결정
                has_unknown = X[col].str.contains('Unknown', na=False).any() if X[col].dtype == object else False
                
                if has_unknown:
                    if col in meaningful_unknown_cols:
                        print(f"      - 'Unknown' 값 유지 (임상적 의미 있음)")
                        if X[col].isnull().any():
                            mode_value = X[col].mode()
                            if not mode_value.empty:
                                fill_value = mode_value[0]
                                X[col] = X[col].fillna(fill_value)
                                print(f"      - 결측치를 '{fill_value}'로 대체")
                    else:
                        print(f"      - 'Unknown' 값을 결측치로 변환 후 대체")
                        X[col] = X[col].replace('Unknown', np.nan)
                        if X[col].isnull().any():
                            mode_value = X[col].mode()
                            if not mode_value.empty:
                                fill_value = mode_value[0]
                                X[col] = X[col].fillna(fill_value)
                                print(f"      - 결측치를 '{fill_value}'로 대체")
                else:
                    if X[col].isnull().any():
                        mode_value = X[col].mode()
                        if not mode_value.empty:
                            fill_value = mode_value[0]
                            X[col] = X[col].fillna(fill_value)
                            print(f"      - 결측치를 '{fill_value}'로 대체")
        
        # 모든 범주형 변수 인코딩
        print("\n🔄 범주형 변수 인코딩:")
        all_categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
        for col in all_categorical_cols:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col].astype(str))
            self.label_encoders[col] = le
            
            if col in meaningful_unknown_cols:
                mapping = dict(zip(le.classes_, le.transform(le.classes_)))
                print(f"   - {col} 인코딩 매핑: {mapping}")
        
        # 모든 특성을 수치형으로 변환
        for col in X.columns:
            X[col] = pd.to_numeric(X[col], errors='coerce')
        
        # 전체 특성에 대해 Imputer 훈련 (메모리 항목[5] 적용)
        print(f"\n📏 전체 특성 Imputer 훈련:")
        print(f"   - 전체 특성 수: {X.shape[1]}")
        
        self.num_imputer = SimpleImputer(strategy='median')
        X_imputed = self.num_imputer.fit_transform(X)
        X = pd.DataFrame(X_imputed, columns=X.columns, index=X.index)
        
        print(f"✅ 전체 특성 Imputer 훈련 완료: {X.shape[1]}개 특성")
        
        # 특성 스케일링
        self.scaler = StandardScaler()
        X_scaled = pd.DataFrame(
            self.scaler.fit_transform(X),
            columns=X.columns,
            index=X.index
        )
        
        self.feature_names = X_scaled.columns.tolist()
        
        # scikit-survival 형식으로 변환
        y_structured = np.array([(event, duration) for event, duration in zip(y_event, y_duration)],
                               dtype=[('event', '?'), ('time', '<f8')])
        
        print("✅ 특성 준비 완료 (CDSS 호환)")
        
        return X_scaled, y_structured, y_duration, y_event, y_effectiveness
    
    def split_data(self, X, y_structured, y_duration, y_event, y_effectiveness):
        """데이터 분할 (메모리 항목[3] 적용)"""
        print("\n✂️  4. 데이터 분할 (훈련:검증:테스트 = 60:20:20)")
        
        # 먼저 훈련+검증 vs 테스트로 분할
        X_temp, X_test, y_temp_struct, y_test_struct, y_temp_dur, y_test_dur, y_temp_event, y_test_event, y_temp_eff, y_test_eff = \
            train_test_split(X, y_structured, y_duration, y_event, y_effectiveness,
                           test_size=0.2, random_state=42, stratify=y_effectiveness)
        
        # 훈련 vs 검증으로 분할
        X_train, X_val, y_train_struct, y_val_struct, y_train_dur, y_val_dur, y_train_event, y_val_event, y_train_eff, y_val_eff = \
            train_test_split(X_temp, y_temp_struct, y_temp_dur, y_temp_event, y_temp_eff,
                           test_size=0.25, random_state=42, stratify=y_temp_eff)
        
        print(f"📊 훈련 세트: {len(X_train)}명 (사망: {y_train_event.sum()}명, 치료효과양호: {y_train_eff.sum()}명)")
        print(f"📊 검증 세트: {len(X_val)}명 (사망: {y_val_event.sum()}명, 치료효과양호: {y_val_eff.sum()}명)")
        print(f"📊 테스트 세트: {len(X_test)}명 (사망: {y_test_event.sum()}명, 치료효과양호: {y_test_eff.sum()}명)")
        print(f"📊 CDSS 테스트: 1명 (별도 보관)")
        
        return (X_train, X_val, X_test, 
                y_train_struct, y_val_struct, y_test_struct,
                y_train_dur, y_val_dur, y_test_dur,
                y_train_event, y_val_event, y_test_event,
                y_train_eff, y_val_eff, y_test_eff)
    
    def train_models(self, X_train, X_val, X_test, 
                    y_train_struct, y_val_struct, y_test_struct,
                    y_train_dur, y_val_dur, y_test_dur,
                    y_train_event, y_val_event, y_test_event,
                    y_train_eff, y_val_eff, y_test_eff):
        """모델 훈련 (균형잡힌 과적합 방지)"""
        print("\n🤖 5. 모델 훈련 (균형잡힌 과적합 방지)")
        
        # 1. Random Survival Forest (생존 예측)
        print("🔄 Random Survival Forest 훈련 중...")
        try:
            rsf_model = RandomSurvivalForest(
                n_estimators=100,
                max_depth=8,
                min_samples_split=15,
                min_samples_leaf=8,
                random_state=42,
                n_jobs=-1
            )
            rsf_model.fit(X_train, y_train_struct)
            
            rsf_wrapper = {
                'model': rsf_model,
                'model_type': 'RandomSurvivalForest',
                'feature_names': self.feature_names,
                'scaler': self.scaler,
                'label_encoders': self.label_encoders,
                'num_imputer': self.num_imputer,
                'training_info': {
                    'n_samples': len(X_train),
                    'n_features': len(self.feature_names),
                    'event_rate': y_train_event.mean()
                }
            }
            
            self.models['RSF'] = rsf_wrapper
            print("✅ Random Survival Forest 훈련 완료")
        except Exception as e:
            print(f"❌ RSF 모델 훈련 실패: {e}")
        
        # 2. Cox 비례위험 모델 (생존 예측)
        print("🔄 Cox 비례위험 모델 훈련 중...")
        try:
            cox_model = CoxPHSurvivalAnalysis(alpha=0.5)
            cox_model.fit(X_train, y_train_struct)
            
            cox_wrapper = {
                'model': cox_model,
                'model_type': 'CoxPHSurvivalAnalysis',
                'feature_names': self.feature_names,
                'scaler': self.scaler,
                'label_encoders': self.label_encoders,
                'num_imputer': self.num_imputer,
                'training_info': {
                    'n_samples': len(X_train),
                    'n_features': len(self.feature_names),
                    'event_rate': y_train_event.mean()
                }
            }
            
            self.models['Cox_Survival'] = cox_wrapper
            print("✅ Cox 생존 모델 훈련 완료")
        except Exception as e:
            print(f"❌ Cox 모델 훈련 실패: {e}")
        
        # 3. Random Forest (균형잡힌 설정)
        print("🔄 Random Forest 균형잡힌 설정으로 훈련 중...")
        try:
            rf_classifier = RandomForestClassifier(
                n_estimators=100,
                max_depth=8,
                min_samples_split=15,
                min_samples_leaf=8,
                max_features='sqrt',
                random_state=42,
                n_jobs=-1
            )
            rf_classifier.fit(X_train, y_train_eff)
            
            rf_wrapper = {
                'model': rf_classifier,
                'model_type': 'RandomForestClassifier',
                'feature_names': self.feature_names,
                'scaler': self.scaler,
                'label_encoders': self.label_encoders,
                'num_imputer': self.num_imputer,
                'class_labels': ['치료효과 불량', '치료효과 양호'],
                'training_info': {
                    'n_samples': len(X_train),
                    'n_features': len(self.feature_names),
                    'class_distribution': np.bincount(y_train_eff),
                    'feature_importance': rf_classifier.feature_importances_
                }
            }
            
            self.models['RF_Treatment'] = rf_wrapper
            print("✅ Random Forest 균형잡힌 훈련 완료")
        except Exception as e:
            print(f"❌ RF 모델 훈련 실패: {e}")
        
        # 4. XGBoost (적절한 정규화)
        print("🔄 XGBoost 적절한 정규화로 훈련 중...")
        try:
            xgb_model = xgb.XGBClassifier(
                n_estimators=100,
                max_depth=5,
                learning_rate=0.1,
                subsample=0.8,
                colsample_bytree=0.8,
                reg_alpha=0.5,
                reg_lambda=0.5,
                random_state=42,
                eval_metric='logloss'
            )
            
            xgb_model.fit(X_train, y_train_eff)
            
            xgb_wrapper = {
                'model': xgb_model,
                'model_type': 'XGBClassifier',
                'feature_names': self.feature_names,
                'scaler': self.scaler,
                'label_encoders': self.label_encoders,
                'num_imputer': self.num_imputer,
                'class_labels': ['치료효과 불량', '치료효과 양호'],
                'training_info': {
                    'n_samples': len(X_train),
                    'n_features': len(self.feature_names),
                    'class_distribution': np.bincount(y_train_eff),
                    'feature_importance': xgb_model.feature_importances_
                }
            }
            
            self.models['XGB_Treatment'] = xgb_wrapper
            print("✅ XGBoost 적절한 정규화 훈련 완료")
        except Exception as e:
            print(f"❌ XGBoost 모델 훈련 실패: {e}")
        
        # 5. LightGBM (과적합 방지 최적화)
        print("🔄 LightGBM 과적합 방지 최적화로 훈련 중...")
        try:
            lgb_model = lgb.LGBMClassifier(
                n_estimators=100,
                max_depth=4,
                learning_rate=0.05,
                num_leaves=15,
                subsample=0.8,
                colsample_bytree=0.8,
                reg_alpha=0.5,
                reg_lambda=0.5,
                min_child_samples=20,
                min_child_weight=0.001,
                random_state=42,
                verbose=-1
            )
            
            lgb_model.fit(
                X_train, y_train_eff,
                eval_set=[(X_val, y_val_eff)],
                callbacks=[lgb.early_stopping(10), lgb.log_evaluation(0)]
            )
            
            lgb_wrapper = {
                'model': lgb_model,
                'model_type': 'LGBMClassifier',
                'feature_names': self.feature_names,
                'scaler': self.scaler,
                'label_encoders': self.label_encoders,
                'num_imputer': self.num_imputer,
                'class_labels': ['치료효과 불량', '치료효과 양호'],
                'training_info': {
                    'n_samples': len(X_train),
                    'n_features': len(self.feature_names),
                    'class_distribution': np.bincount(y_train_eff),
                    'feature_importance': lgb_model.feature_importances_
                }
            }
            
            self.models['LGB_Treatment'] = lgb_wrapper
            print("✅ LightGBM 과적합 방지 훈련 완료")
        except Exception as e:
            print(f"❌ LightGBM 모델 훈련 실패: {e}")
        
        print(f"\n🎯 균형잡힌 {len(self.models)}개 모델 훈련 완료")
        
        return True
    
    def evaluate_models(self, X_train, X_val, X_test,
                       y_train_struct, y_val_struct, y_test_struct,
                       y_train_dur, y_val_dur, y_test_dur,
                       y_train_event, y_val_event, y_test_event,
                       y_train_eff, y_val_eff, y_test_eff):
        """모델 평가"""
        print("\n📈 6. 모델 평가")
        
        datasets = {
            'Train': (X_train, y_train_struct, y_train_dur, y_train_event, y_train_eff),
            'Validation': (X_val, y_val_struct, y_val_dur, y_val_event, y_val_eff),
            'Test': (X_test, y_test_struct, y_test_dur, y_test_event, y_test_eff)
        }
        
        for model_name, model_wrapper in self.models.items():
            print(f"\n🔍 {model_name} 모델 평가:")
            self.results[model_name] = {}
            
            for dataset_name, (X, y_struct, y_dur, y_event, y_eff) in datasets.items():
                try:
                    actual_model = model_wrapper['model']
                    model_type = model_wrapper['model_type']
                    
                    if model_type in ['RandomSurvivalForest', 'CoxPHSurvivalAnalysis']:
                        # 생존 모델 평가
                        risk_scores = actual_model.predict(X)
                        c_index = concordance_index_censored(y_struct['event'], y_struct['time'], risk_scores)[0]
                        self.results[model_name][dataset_name] = {'c_index': c_index}
                        print(f"   {dataset_name}: C-index = {c_index:.3f}")
                    
                    else:
                        # 분류 모델 평가 (치료 효과)
                        y_pred = actual_model.predict(X)
                        accuracy = (y_pred == y_eff).mean()
                        
                        try:
                            y_proba = actual_model.predict_proba(X)
                            auc_score = roc_auc_score(y_eff, y_proba[:, 1])
                        except:
                            auc_score = np.nan
                        
                        self.results[model_name][dataset_name] = {
                            'accuracy': accuracy,
                            'auc': auc_score
                        }
                        print(f"   {dataset_name}: Accuracy = {accuracy:.3f}, AUC = {auc_score:.3f}")
                    
                except Exception as e:
                    print(f"   ❌ {dataset_name} 평가 실패: {e}")
                    self.results[model_name][dataset_name] = {'error': str(e)}
        
        return True
    
    def explain_models(self, X_train, X_test):
        """XAI 모델 설명 생성"""
        print("\n🔍 XAI 모델 설명 생성")
        
        # SHAP 설명기 초기화
        print("🔄 SHAP 설명 생성 중...")
        tree_models = ['RF_Treatment', 'XGB_Treatment', 'LGB_Treatment']
        
        for model_name in tree_models:
            if model_name in self.models:
                try:
                    X_test_sample = X_test.iloc[:50]
                    
                    actual_model = self.models[model_name]['model']
                    
                    explainer = shap.TreeExplainer(actual_model)
                    shap_values = explainer.shap_values(X_test_sample)
                    
                    self.shap_explainers[model_name] = explainer
                    self.shap_values[model_name] = shap_values
                    print(f"✅ {model_name} SHAP 설명 생성 완료")
                except Exception as e:
                    print(f"❌ {model_name} SHAP 실패: {e}")
        
        # LIME 설명기 초기화
        print("\n🔄 LIME 설명 생성 중...")
        classification_models = ['RF_Treatment', 'XGB_Treatment', 'LGB_Treatment']
        
        for model_name in classification_models:
            if model_name in self.models:
                try:
                    actual_model = self.models[model_name]['model']
                    class_labels = self.models[model_name]['class_labels']
                    
                    explainer = lime_tabular.LimeTabularExplainer(
                        training_data=X_train.values,
                        feature_names=self.feature_names,
                        class_names=class_labels,
                        mode='classification',
                        discretize_continuous=True
                    )
                    
                    self.lime_explainers[model_name] = {
                        'explainer': explainer,
                        'predict_fn': actual_model.predict_proba
                    }
                    print(f"✅ {model_name} LIME 설명기 생성 완료")
                except Exception as e:
                    print(f"❌ {model_name} LIME 실패: {e}")
        
        return True
    
    def generate_xai_visualizations(self, X_test, sample_index=0):
        """XAI 시각화 생성 및 저장"""
        print("\n📊 XAI 시각화 생성")
        
        # SHAP 시각화
        shap_figures = []
        for model_name in self.shap_explainers:
            try:
                print(f"🔄 {model_name} SHAP 시각화 생성 중...")
                
                shap_vals = self.shap_values[model_name]
                
                # 이진 분류 SHAP 값 처리
                if isinstance(shap_vals, list) and len(shap_vals) == 2:
                    shap_vals_to_plot = shap_vals[1]
                    expected_val = self.shap_explainers[model_name].expected_value[1]
                elif hasattr(shap_vals, 'shape') and shap_vals.ndim == 3:
                    shap_vals_to_plot = shap_vals[:, :, 1]
                    expected_val = (self.shap_explainers[model_name].expected_value[1] 
                                if hasattr(self.shap_explainers[model_name].expected_value, '__len__') 
                                else self.shap_explainers[model_name].expected_value)
                else:
                    shap_vals_to_plot = shap_vals
                    expected_val = (self.shap_explainers[model_name].expected_value 
                                if not hasattr(self.shap_explainers[model_name].expected_value, '__len__')
                                else self.shap_explainers[model_name].expected_value[0])
                
                # Summary plot
                plt.figure(figsize=(10,6))
                shap.summary_plot(shap_vals_to_plot, X_test.iloc[:50], 
                                feature_names=self.feature_names,
                                plot_type="bar", show=False)
                plt.title(f"{model_name} 특성 중요도 (SHAP)")
                shap_summary_path = f"shap_summary_{model_name}.png"
                plt.savefig(shap_summary_path, bbox_inches='tight')
                plt.close()
                
                # Individual waterfall plot
                plt.figure(figsize=(12,6))
                
                if shap_vals_to_plot.ndim == 2:
                    individual_shap = shap_vals_to_plot[sample_index, :]
                else:
                    individual_shap = shap_vals_to_plot
                
                if hasattr(expected_val, '__len__') and len(expected_val) > 0:
                    base_value = float(expected_val[0]) if hasattr(expected_val[0], '__float__') else 0.0
                else:
                    base_value = float(expected_val) if hasattr(expected_val, '__float__') else 0.0
                
                try:
                    shap.waterfall_plot(
                        shap.Explanation(
                            values=individual_shap.astype(float),
                            base_values=base_value,
                            data=X_test.iloc[sample_index].values.astype(float),
                            feature_names=self.feature_names
                        ),
                        show=False
                    )
                    plt.title(f"{model_name} 개별 설명 (샘플 {sample_index})")
                    shap_waterfall_path = f"shap_waterfall_{model_name}_{sample_index}.png"
                    plt.savefig(shap_waterfall_path, bbox_inches='tight')
                    plt.close()
                    
                    shap_figures.extend([shap_summary_path, shap_waterfall_path])
                    
                except Exception as waterfall_error:
                    print(f"⚠️ Waterfall plot 실패, Force plot으로 대체: {waterfall_error}")
                    
                    shap.force_plot(
                        base_value,
                        individual_shap,
                        X_test.iloc[sample_index],
                        feature_names=self.feature_names,
                        matplotlib=True,
                        show=False
                    )
                    plt.title(f"{model_name} 개별 설명 - Force Plot (샘플 {sample_index})")
                    shap_force_path = f"shap_force_{model_name}_{sample_index}.png"
                    plt.savefig(shap_force_path, bbox_inches='tight')
                    plt.close()
                    
                    shap_figures.extend([shap_summary_path, shap_force_path])
                
                print(f"✅ {model_name} SHAP 시각화 완료")
                
            except Exception as e:
                print(f"❌ {model_name} SHAP 시각화 실패: {e}")
        
        # LIME 시각화
        lime_figures = []
        for model_name in self.lime_explainers:
            try:
                lime_data = self.lime_explainers[model_name]
                exp = lime_data['explainer'].explain_instance(
                    X_test.iloc[sample_index].values,
                    lime_data['predict_fn'],
                    num_features=5
                )
                lime_path = f"lime_explanation_{model_name}_{sample_index}.png"
                fig = exp.as_pyplot_figure()
                plt.title(f"{model_name} LIME 설명 (샘플 {sample_index})")
                plt.savefig(lime_path, bbox_inches='tight')
                plt.close()
                lime_figures.append(lime_path)
                print(f"✅ {model_name} LIME 시각화 완료")
            except Exception as e:
                print(f"❌ {model_name} LIME 시각화 실패: {e}")
        
        return shap_figures, lime_figures
    
    def preprocess_holdout_patient(self):
        """Holdout 환자 데이터 전처리 (더 안전한 방법)"""
        print("\n🔧 Holdout 환자 전처리 시작")
        
        # holdout 환자의 특성 데이터만 추출
        feature_cols = [col for col in self.holdout_patient.columns 
                       if col not in ['vital_status', 'days_to_death', 'days_to_last_follow_up', 
                                     'event', 'duration', 'treatment_effectiveness']]
        
        patient_raw = self.holdout_patient[feature_cols].copy()
        print(f"🔍 원본 환자 특성: {len(patient_raw.columns)}개")
        print(f"🔍 모델 훈련 특성: {len(self.feature_names)}개")
        
        # 모델 훈련 시 사용한 특성명과 정확히 일치하는 DataFrame 생성
        patient_processed = pd.DataFrame(index=patient_raw.index)
        
        for feature_name in self.feature_names:
            if feature_name in patient_raw.columns:
                patient_processed[feature_name] = patient_raw[feature_name].copy()
                print(f"✅ {feature_name}: 원본 데이터 사용")
            else:
                patient_processed[feature_name] = 0.0
                print(f"⚠️ {feature_name}: 기본값(0.0) 설정")
        
        # 범주형 변수 전처리
        print("\n🔄 범주형 변수 전처리:")
        for col, encoder in self.label_encoders.items():
            if col in patient_processed.columns:
                try:
                    original_value = patient_processed[col].iloc[0]
                    print(f"   - {col}: 원본값 = {original_value}")
                    
                    if pd.isna(original_value) or original_value == 'NA':
                        encoded_value = encoder.transform([encoder.classes_[0]])[0]
                        patient_processed[col] = float(encoded_value)
                        print(f"     → 결측치를 '{encoder.classes_[0]}' ({encoded_value})로 대체")
                    else:
                        str_value = str(original_value)
                        if str_value in encoder.classes_:
                            encoded_value = encoder.transform([str_value])[0]
                            patient_processed[col] = float(encoded_value)
                            print(f"     → 인코딩: '{str_value}' → {encoded_value}")
                        else:
                            encoded_value = encoder.transform([encoder.classes_[0]])[0]
                            patient_processed[col] = float(encoded_value)
                            print(f"     → 새로운 값 '{str_value}'을 '{encoder.classes_[0]}' ({encoded_value})로 대체")
                            
                except Exception as e:
                    print(f"     ❌ {col} 인코딩 실패: {e}")
                    patient_processed[col] = 0.0
        
        # 모든 컬럼을 수치형으로 변환
        print("\n🔢 데이터 타입 변환:")
        for col in patient_processed.columns:
            try:
                if pd.api.types.is_numeric_dtype(patient_processed[col]):
                    print(f"   - {col}: 이미 숫자형")
                else:
                    patient_processed[col] = pd.to_numeric(patient_processed[col], errors='coerce')
                    print(f"   - {col}: 숫자형으로 변환")
                
                if patient_processed[col].isnull().any():
                    patient_processed[col] = patient_processed[col].fillna(0.0)
                    print(f"   - {col}: 결측값을 0.0으로 대체")
                    
            except Exception as e:
                print(f"⚠️ {col} 수치형 변환 실패: {e}")
                patient_processed[col] = 0.0
        
        # 특성 순서를 훈련 시와 정확히 일치시키기
        patient_processed = patient_processed[self.feature_names]
        print(f"✅ 특성 순서 정렬 완료: {patient_processed.shape}")
        print(f"✅ 특성명 확인: {list(patient_processed.columns) == self.feature_names}")
        
        # Imputer 적용 (안전한 방법)
        if hasattr(self, 'num_imputer') and self.num_imputer is not None:
            try:
                print(f"\n📏 Imputer 적용 (안전한 방법):")
                print(f"   - 입력 형태: {patient_processed.shape}")
                
                if hasattr(self.num_imputer, 'n_features_in_'):
                    expected_features = self.num_imputer.n_features_in_
                    print(f"   - Imputer 기대 특성 수: {expected_features}")
                    
                    if patient_processed.shape[1] == expected_features:
                        patient_values = patient_processed.values
                        imputed_values = self.num_imputer.transform(patient_values)
                        patient_processed = pd.DataFrame(
                            imputed_values,
                            columns=self.feature_names,
                            index=patient_processed.index
                        )
                        print(f"✅ Imputer 적용 완료")
                    else:
                        print(f"⚠️ 특성 수 불일치: 입력 {patient_processed.shape[1]} vs 기대 {expected_features}")
                        print(f"⚠️ Imputer 건너뜀 - 결측값은 이미 처리됨")
                else:
                    print(f"⚠️ Imputer 특성 정보 없음 - 안전하게 건너뜀")
                    
            except Exception as e:
                print(f"⚠️ Imputer 적용 중 오류 발생, 건너뜀: {e}")
        else:
            print(f"ℹ️ Imputer가 없거나 None - 건너뜀")
        
        # Scaler 적용 (안전한 방법)
        if hasattr(self, 'scaler') and self.scaler is not None:
            try:
                print(f"\n📏 스케일링 적용 (안전한 방법):")
                print(f"   - 입력 형태: {patient_processed.shape}")
                
                if hasattr(self.scaler, 'n_features_in_'):
                    expected_features = self.scaler.n_features_in_
                    print(f"   - Scaler 기대 특성 수: {expected_features}")
                    
                    if patient_processed.shape[1] == expected_features:
                        patient_values = patient_processed.values
                        scaled_values = self.scaler.transform(patient_values)
                        patient_features_scaled = pd.DataFrame(
                            scaled_values,
                            columns=self.feature_names,
                            index=patient_processed.index
                        )
                        print(f"✅ 스케일링 완료")
                        print(f"✅ 최종 특성 형태: {patient_features_scaled.shape}")
                        return patient_features_scaled
                    else:
                        print(f"⚠️ 특성 수 불일치: 입력 {patient_processed.shape[1]} vs 기대 {expected_features}")
                        print(f"⚠️ 스케일링 없이 반환")
                        return patient_processed
                else:
                    print(f"⚠️ Scaler 특성 정보 없음 - 안전하게 건너뜀")
                    return patient_processed
                    
            except Exception as e:
                print(f"❌ 스케일링 중 오류 발생: {e}")
                print(f"⚠️ 스케일링 없이 반환")
                return patient_processed
        else:
            print(f"ℹ️ Scaler가 없거나 None - 건너뜀")
            return patient_processed
    
    def test_cdss_compatibility(self):
        """CDSS 호환성 테스트"""
        print("\n🔬 CDSS 호환성 테스트")
        
        try:
            # holdout 환자 데이터 전처리
            holdout_features = self.preprocess_holdout_patient()
            
            # 각 모델로 예측 수행
            predictions = {}
            
            for model_name, model_wrapper in self.models.items():
                try:
                    print(f"\n🔄 {model_name} 모델 예측 중...")
                    
                    actual_model = model_wrapper['model']
                    model_type = model_wrapper['model_type']
                    
                    print(f"   - 모델 타입: {model_type}")
                    print(f"   - 입력 특성 수: {holdout_features.shape[1]}")
                    
                    if model_type in ['RandomSurvivalForest', 'CoxPHSurvivalAnalysis']:
                        # 생존 모델
                        pred = actual_model.predict(holdout_features)[0]
                        predictions[model_name] = {
                            'type': 'survival',
                            'risk_score': pred
                        }
                        print(f"✅ {model_name}: 위험도 점수 = {pred:.4f}")
                    else:
                        # 분류 모델
                        pred_class = actual_model.predict(holdout_features)[0]
                        pred_proba = actual_model.predict_proba(holdout_features)[0]
                        class_labels = model_wrapper['class_labels']
                        
                        predictions[model_name] = {
                            'type': 'classification',
                            'class': pred_class,
                            'class_name': class_labels[pred_class],
                            'probabilities': pred_proba
                        }
                        
                        print(f"✅ {model_name}: {class_labels[pred_class]} (확률: {pred_proba[pred_class]:.3f})")
                    
                except Exception as e:
                    print(f"❌ {model_name} 예측 실패: {e}")
                    predictions[model_name] = {'error': str(e)}
            
            return predictions
            
        except Exception as e:
            print(f"❌ CDSS 호환성 테스트 실패: {e}")
            return {}
    
    def save_models_for_cdss(self):
        """CDSS 호환 모델 저장"""
        print("\n💾 CDSS 호환 모델 저장")
        
        # 전체 파이프라인을 하나의 객체로 저장
        cdss_pipeline = {
            'models': self.models,
            'holdout_patient': self.holdout_patient,
            'metadata': {
                'created_date': datetime.now().isoformat(),
                'model_version': '4.0',
                'description': 'TCGA-LIHC 간암 치료 효과 예측 모델 (CDSS 호환 + 과적합 방지)',
                'class_labels': ['치료효과 불량', '치료효과 양호']
            }
        }
        
        # 개별 모델도 저장
        for model_name, model_wrapper in self.models.items():
            try:
                filename = f"cdss_liver_cancer_treatment_{model_name.lower()}_model.pkl"
                with open(filename, 'wb') as f:
                    pickle.dump(model_wrapper, f)
                print(f"✅ {model_name} 모델 저장: {filename}")
            except Exception as e:
                print(f"❌ {model_name} 모델 저장 실패: {e}")
        
        # 전체 파이프라인 저장
        try:
            pipeline_filename = "cdss_liver_cancer_treatment_complete_pipeline.pkl"
            with open(pipeline_filename, 'wb') as f:
                pickle.dump(cdss_pipeline, f)
            print(f"✅ 전체 파이프라인 저장: {pipeline_filename}")
        except Exception as e:
            print(f"❌ 파이프라인 저장 실패: {e}")
        
        return True
    
    def plot_results(self, X_test, y_test_dur, y_test_event, y_test_eff):
        """결과 시각화 (치료 효과 예측 특화)"""
        print("\n📊 7. 결과 시각화")
        
        fig = plt.figure(figsize=(25, 35))
        gs = fig.add_gridspec(5, 3)
        axes = [
            fig.add_subplot(gs[0, 0]),  # 치료효과별 생존 곡선
            fig.add_subplot(gs[0, 1]),  # 치료효과 분포
            fig.add_subplot(gs[0, 2]),  # 생존 모델 성능
            fig.add_subplot(gs[1, 0]),  # 치료효과 분류 성능
            fig.add_subplot(gs[1, 1]),  # AUC 성능
            fig.add_subplot(gs[1, 2]),  # 특성 중요도 (XGBoost)
            fig.add_subplot(gs[2, 0]),  # 특성 중요도 (LightGBM)
            fig.add_subplot(gs[2, 1]),  # 치료효과별 생존율
            fig.add_subplot(gs[2, 2]),  # CDSS 테스트 결과
            fig.add_subplot(gs[3, :]),  # SHAP 시각화
            fig.add_subplot(gs[4, :])   # LIME 시각화
        ]
        
        fig.suptitle('간암 치료 효과 예측 모델 분석 결과 (CDSS 호환 + XAI + 과적합 방지)', fontsize=16, fontweight='bold')
        
        # 1. 치료효과별 생존 곡선
        print("🔍 치료효과별 생존 곡선 생성 중...")
        for eff_level in [0, 1]:
            mask = (y_test_eff == eff_level)
            if mask.sum() > 5:
                kmf = KaplanMeierFitter()
                label = '치료효과 양호' if eff_level == 1 else '치료효과 불량'
                kmf.fit(y_test_dur[mask], y_test_event[mask], label=f'{label} (n={mask.sum()})')
                kmf.plot_survival_function(ax=axes[0])
        
        axes[0].set_title('치료효과별 생존 곡선 (Kaplan-Meier)')
        axes[0].set_ylabel('생존 확률')
        axes[0].set_xlabel('시간 (일)')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # 2. 치료효과 분포
        eff_counts = pd.Series(y_test_eff).value_counts().sort_index()
        colors = ['lightcoral', 'lightgreen']
        labels = ['치료효과 불량', '치료효과 양호']
        bars = axes[1].bar([labels[i] for i in eff_counts.index], 
                          eff_counts.values, color=colors)
        axes[1].set_title('테스트 세트 치료효과 분포')
        axes[1].set_ylabel('환자 수')
        
        for bar, value in zip(bars, eff_counts.values):
            axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                        f'{value}명', ha='center', va='bottom', fontweight='bold')
        
        # 3. 생존 모델 성능 비교
        survival_models = ['RSF', 'Cox_Survival']
        survival_c_indices = []
        survival_names = []
        
        for model_name in survival_models:
            if model_name in self.results and 'Test' in self.results[model_name]:
                c_index = self.results[model_name]['Test'].get('c_index', np.nan)
                if not np.isnan(c_index):
                    survival_c_indices.append(c_index)
                    survival_names.append(model_name)
        
        if survival_c_indices:
            bars = axes[2].bar(survival_names, survival_c_indices, 
                             color=['skyblue', 'lightcoral'][:len(survival_names)])
            axes[2].set_title('생존 모델 성능 (C-index)')
            axes[2].set_ylabel('C-index')
            axes[2].set_ylim(0.5, 1.0)
            axes[2].grid(True, alpha=0.3, axis='y')
            
            for bar, value in zip(bars, survival_c_indices):
                axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                            f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 4. 치료효과 분류 모델 성능 (Accuracy)
        classification_models = ['RF_Treatment', 'XGB_Treatment', 'LGB_Treatment']
        classification_accuracies = []
        classification_names = []
        
        for model_name in classification_models:
            if model_name in self.results and 'Test' in self.results[model_name]:
                accuracy = self.results[model_name]['Test'].get('accuracy', np.nan)
                if not np.isnan(accuracy):
                    classification_accuracies.append(accuracy)
                    classification_names.append(model_name)
        
        if classification_accuracies:
            bars = axes[3].bar(classification_names, classification_accuracies, 
                             color=['lightgreen', 'orange', 'purple'][:len(classification_names)])
            axes[3].set_title('치료효과 분류 모델 성능 (Accuracy)')
            axes[3].set_ylabel('Accuracy')
            axes[3].set_ylim(0, 1.0)
            axes[3].grid(True, alpha=0.3, axis='y')
            
            for bar, value in zip(bars, classification_accuracies):
                axes[3].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                            f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 5. AUC 성능
        classification_aucs = []
        auc_names = []
        
        for model_name in classification_models:
            if model_name in self.results and 'Test' in self.results[model_name]:
                auc = self.results[model_name]['Test'].get('auc', np.nan)
                if not np.isnan(auc):
                    classification_aucs.append(auc)
                    auc_names.append(model_name)
        
        if classification_aucs:
            bars = axes[4].bar(auc_names, classification_aucs, 
                             color=['lightgreen', 'orange', 'purple'][:len(auc_names)])
            axes[4].set_title('치료효과 분류 모델 성능 (AUC)')
            axes[4].set_ylabel('AUC')
            axes[4].set_ylim(0, 1.0)
            axes[4].grid(True, alpha=0.3, axis='y')
            
            for bar, value in zip(bars, classification_aucs):
                axes[4].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                            f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 6-7. 특성 중요도 시각화
        for idx, model_name in enumerate(['XGB_Treatment', 'LGB_Treatment']):
            ax_idx = 5 + idx
            if model_name in self.models:
                try:
                    model_wrapper = self.models[model_name]
                    importance = model_wrapper['training_info']['feature_importance']
                    
                    feature_importance_df = pd.DataFrame({
                        'feature': self.feature_names,
                        'importance': importance
                    }).sort_values('importance', ascending=True).tail(10)
                    
                    color = 'orange' if 'XGB' in model_name else 'purple'
                    bars = axes[ax_idx].barh(range(len(feature_importance_df)), 
                                           feature_importance_df['importance'],
                                           color=color)
                    
                    axes[ax_idx].set_yticks(range(len(feature_importance_df)))
                    axes[ax_idx].set_yticklabels(feature_importance_df['feature'], fontsize=10)
                    axes[ax_idx].set_title(f'특성 중요도 ({model_name})', fontsize=12, fontweight='bold')
                    axes[ax_idx].set_xlabel('중요도')
                    axes[ax_idx].grid(True, alpha=0.3, axis='x')
                    
                except Exception as e:
                    print(f"❌ {model_name} 특성 중요도 시각화 실패: {e}")
        
        # 8. 치료효과별 생존율 요약
        survival_summary = []
        time_points = [365, 1095, 1825]  # 1년, 3년, 5년
        
        for eff_level in [0, 1]:
            mask = (y_test_eff == eff_level)
            if mask.sum() > 5:
                kmf = KaplanMeierFitter()
                kmf.fit(y_test_dur[mask], y_test_event[mask])
                
                survival_rates = []
                for time_point in time_points:
                    try:
                        survival_rate = kmf.survival_function_at_times(time_point).values[0]
                        survival_rates.append(survival_rate * 100)
                    except:
                        survival_rates.append(np.nan)
                
                survival_summary.append(survival_rates)
            else:
                survival_summary.append([np.nan, np.nan, np.nan])
        
        # 생존율 히트맵
        survival_df = pd.DataFrame(survival_summary, 
                                  index=['치료효과 불량', '치료효과 양호'],
                                  columns=['1년', '3년', '5년'])
        
        sns.heatmap(survival_df, annot=True, fmt='.1f', cmap='RdYlGn', 
                   ax=axes[7], cbar_kws={'label': '생존율 (%)'})
        axes[7].set_title('치료효과별 생존율 요약')
        axes[7].set_ylabel('치료효과 그룹')
        
        # 9. CDSS 테스트 결과
        cdss_results = self.test_cdss_compatibility()
        if cdss_results:
            model_names_cdss = []
            predictions_cdss = []
            
            for model_name, result in cdss_results.items():
                if 'type' in result:
                    model_names_cdss.append(model_name)
                    if result['type'] == 'classification':
                        predictions_cdss.append(result['class'])
                    else:
                        predictions_cdss.append(result['risk_score'])
            
            if model_names_cdss:
                bars = axes[8].bar(model_names_cdss, predictions_cdss, 
                                 color=['skyblue', 'lightcoral', 'lightgreen', 'orange', 'purple'][:len(model_names_cdss)])
                axes[8].set_title('CDSS 호환성 테스트\n(Holdout 환자 예측)')
                axes[8].set_ylabel('예측값')
                axes[8].grid(True, alpha=0.3, axis='y')
                
                for bar, value in zip(bars, predictions_cdss):
                    axes[8].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                               f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 10-11. XAI 시각화 로드
        try:
            model_name = 'XGB_Treatment' if 'XGB_Treatment' in self.shap_explainers else 'RF_Treatment'
            if model_name:
                img = plt.imread(f"shap_summary_{model_name}.png")
                axes[9].imshow(img)
                axes[9].axis('off')
                axes[9].set_title('SHAP 전역 설명 (치료효과 예측)', fontsize=12)
        except Exception as e:
            axes[9].text(0.5, 0.5, 'SHAP 시각화 불러오기 실패', 
                        ha='center', va='center', transform=axes[9].transAxes,
                        fontsize=12, fontweight='bold')
        
        try:
            model_name = 'XGB_Treatment' if 'XGB_Treatment' in self.lime_explainers else 'RF_Treatment'
            if model_name:
                img = plt.imread(f"lime_explanation_{model_name}_0.png")
                axes[10].imshow(img)
                axes[10].axis('off')
                axes[10].set_title('LIME 개별 설명 (첫 번째 환자)', fontsize=12)
        except Exception as e:
            axes[10].text(0.5, 0.5, 'LIME 시각화 불러오기 실패', 
                         ha='center', va='center', transform=axes[10].transAxes,
                         fontsize=12, fontweight='bold')
        
        plt.tight_layout()
        save_path = "liver_cancer_treatment_effect_prediction_cdss_xai_overfitting_fixed.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"📁 시각화 결과 저장: {save_path}")
        
        return True
    
    def generate_report(self):
        """결과 보고서 생성"""
        print("\n📋 8. 결과 보고서 생성")
        
        report = []
        report.append("="*60)
        report.append("간암 환자 치료 효과 예측 모델 분석 결과 (XAI + CDSS 호환 + 과적합 방지)")
        report.append("="*60)
        report.append(f"분석 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        report.append(f"데이터 경로: {self.data_path}")
        report.append("")
        
        # 데이터 요약
        report.append("📊 데이터 요약")
        report.append("-" * 30)
        report.append(f"총 환자 수: {len(self.processed_df) + 1}")  # holdout 포함
        report.append(f"모델 훈련용: {len(self.processed_df)}명")
        report.append(f"CDSS 테스트용: 1명")
        report.append(f"사망률: {self.processed_df['event'].mean()*100:.1f}%")
        report.append(f"중간 생존 시간: {self.processed_df['duration'].median():.0f}일")
        report.append(f"치료효과 양호율: {self.processed_df['treatment_effectiveness'].mean()*100:.1f}%")
        report.append(f"사용된 특성 수: {len(self.feature_names)}")
        report.append("")
        
        # 모델 성능
        report.append("🤖 모델 성능")
        report.append("-" * 30)
        
        # 생존 모델 성능
        report.append("\n생존 예측 모델 (C-index):")
        survival_models = ['RSF', 'Cox_Survival']
        for model_name in survival_models:
            if model_name in self.results:
                for dataset in ['Train', 'Validation', 'Test']:
                    if dataset in self.results[model_name]:
                        metrics = self.results[model_name][dataset]
                        if 'c_index' in metrics:
                            report.append(f"  {model_name} {dataset}: C-index = {metrics['c_index']:.3f}")
        
        # 치료효과 분류 모델 성능
        report.append("\n치료효과 분류 모델 (Accuracy):")
        classification_models = ['RF_Treatment', 'XGB_Treatment', 'LGB_Treatment']
        for model_name in classification_models:
            if model_name in self.results:
                for dataset in ['Train', 'Validation', 'Test']:
                    if dataset in self.results[model_name]:
                        metrics = self.results[model_name][dataset]
                        if 'accuracy' in metrics:
                            report.append(f"  {model_name} {dataset}: Accuracy = {metrics['accuracy']:.3f}")
                        if 'auc' in metrics and not np.isnan(metrics['auc']):
                            report.append(f"  {model_name} {dataset}: AUC = {metrics['auc']:.3f}")
        
        # 과적합 분석
        report.append("\n과적합 분석:")
        for model_name in classification_models:
            if model_name in self.results:
                train_acc = self.results[model_name].get('Train', {}).get('accuracy', np.nan)
                test_acc = self.results[model_name].get('Test', {}).get('accuracy', np.nan)
                if not np.isnan(train_acc) and not np.isnan(test_acc):
                    overfitting_gap = train_acc - test_acc
                    report.append(f"  {model_name}: 과적합 정도 = {overfitting_gap:.3f} ({'심각' if overfitting_gap > 0.1 else '양호' if overfitting_gap < 0.05 else '보통'})")
        
        # 최고 성능 모델
        test_performances = {}
        for model_name in self.results:
            if 'Test' in self.results[model_name]:
                metrics = self.results[model_name]['Test']
                if 'c_index' in metrics:
                    test_performances[f"{model_name}_C-index"] = metrics['c_index']
                elif 'accuracy' in metrics:
                    test_performances[f"{model_name}_Accuracy"] = metrics['accuracy']
        
        if test_performances:
            best_model = max(test_performances, key=test_performances.get)
            report.append(f"\n🏆 최고 성능 모델: {best_model} (점수: {test_performances[best_model]:.3f})")
        
        report.append("")
        report.append("ℹ️  특징:")
        report.append("   - CDSS 호환 치료 효과 예측 모델")
        report.append("   - XAI 설명 가능성 포함 (SHAP, LIME)")
        report.append("   - Holdout 환자로 실제 예측 테스트")
        report.append("   - 과적합 방지 기법 적용")
        report.append("   - 생존 예측 + 치료 효과 분류 통합")
        report.append("")
        report.append("="*60)
        
        # 보고서 저장
        report_text = "\n".join(report)
        with open("liver_cancer_treatment_effect_prediction_cdss_overfitting_fixed_report.txt", "w", encoding="utf-8") as f:
            f.write(report_text)
        
        print("📁 보고서 저장: liver_cancer_treatment_effect_prediction_cdss_overfitting_fixed_report.txt")
        print("\n" + report_text)
        
        return report_text
    
    def run_complete_analysis(self):
        """전체 분석 실행 (XAI + CDSS 호환 + 과적합 방지)"""
        print("🎯 간암 치료 효과 예측 모델 전체 분석 시작 (XAI + CDSS 호환 + 과적합 방지)")
        
        try:
            # 1. 데이터 로드
            if not self.load_and_explore_data():
                return False
            
            # 2. 데이터 전처리
            if not self.preprocess_data():
                return False
            
            # 3. 특성 준비 (CDSS 호환)
            X, y_structured, y_duration, y_event, y_effectiveness = self.prepare_features()
            
            # 4. 데이터 분할
            (X_train, X_val, X_test, 
             y_train_struct, y_val_struct, y_test_struct,
             y_train_dur, y_val_dur, y_test_dur,
             y_train_event, y_val_event, y_test_event,
             y_train_eff, y_val_eff, y_test_eff) = self.split_data(X, y_structured, y_duration, y_event, y_effectiveness)
            
            # 5. 모델 훈련 (과적합 방지)
            if not self.train_models(X_train, X_val, X_test,
                                   y_train_struct, y_val_struct, y_test_struct,
                                   y_train_dur, y_val_dur, y_test_dur,
                                   y_train_event, y_val_event, y_test_event,
                                   y_train_eff, y_val_eff, y_test_eff):
                return False
            
            # 6. 모델 평가
            if not self.evaluate_models(X_train, X_val, X_test,
                                       y_train_struct, y_val_struct, y_test_struct,
                                       y_train_dur, y_val_dur, y_test_dur,
                                       y_train_event, y_val_event, y_test_event,
                                       y_train_eff, y_val_eff, y_test_eff):
                return False
            
            # 7. XAI 설명 생성
            self.explain_models(X_train, X_test)
            
            # 8. XAI 시각화 생성
            self.generate_xai_visualizations(X_test)
            
            # 9. 시각화 (XAI + CDSS 포함)
            if not self.plot_results(X_test, y_test_dur, y_test_event, y_test_eff):
                return False
            
            # 10. 보고서 생성
            self.generate_report()
            
            # 11. CDSS 호환 모델 저장
            self.save_models_for_cdss()
            
            print("\n🎉 전체 분석 완료!")
            print(f"⏰ 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            
            return True
            
        except Exception as e:
            print(f"\n❌ 분석 중 오류 발생: {e}")
            import traceback
            traceback.print_exc()
            return False

# 실행
if __name__ == "__main__":
    # 데이터 경로
    data_path = r"C:\Users\02\Documents\GDCdata_liver\clinical_data_liver.csv"
    
    # 분석 실행
    treatment_predictor = LiverCancerTreatmentEffectPredictorCDSS(data_path)
    success = treatment_predictor.run_complete_analysis()
    
    if success:
        print("\n✨ 모든 분석이 성공적으로 완료되었습니다!")
        print("📁 생성된 파일들:")
        print("   - liver_cancer_treatment_effect_prediction_cdss_xai_overfitting_fixed.png (시각화 결과)")
        print("   - liver_cancer_treatment_effect_prediction_cdss_overfitting_fixed_report.txt (분석 보고서)")
        print("   - cdss_liver_cancer_treatment_*_model.pkl (CDSS 호환 개별 모델)")
        print("   - cdss_liver_cancer_treatment_complete_pipeline.pkl (전체 파이프라인)")
        print("   - shap_*.png (SHAP 설명)")
        print("   - lime_*.png (LIME 설명)")
        print("\n🔬 주요 개선사항:")
        print("   - 과적합 방지 기법 적용")
        print("   - 균형잡힌 모델 하이퍼파라미터")
        print("   - 안전한 CDSS 호환성")
        print("   - XAI 설명 가능성 포함")
    else:
        print("\n❌ 분석 중 문제가 발생했습니다. 로그를 확인해주세요.")


✅ 한글 폰트 설정 완료
🚀 간암 치료 효과 예측 모델 초기화 (CDSS 호환 + XAI + 과적합 방지)
📁 데이터 경로: C:\Users\02\Documents\GDCdata_liver\clinical_data_liver.csv
⏰ 시작 시간: 2025-06-16 12:42:43
🎯 간암 치료 효과 예측 모델 전체 분석 시작 (XAI + CDSS 호환 + 과적합 방지)

📊 1. 데이터 로드 및 탐색
✅ 데이터 로드 성공: 377행 × 87열
📈 데이터 기본 정보:
   - 총 환자 수: 377
   - 총 컬럼 수: 87
   - 생존 환자: 245명
   - 사망 환자: 132명
   - 사망률: 35.0%

🔧 2. 데이터 전처리
✅ 사용 가능한 컬럼: 20개
🔄 생존 변수 생성 중...
✅ 유효한 생존 데이터: 372명
   - 사망 이벤트: 132건
   - 중간 생존 시간: 602일
   - 치료 효과 양호: 188명

📋 결측값 분석:
   - days_to_death: 240개 (64.5%)
   - child_pugh_classification: 47개 (12.6%)
   - ishak_fibrosis_score: 96개 (25.8%)
   - ajcc_pathologic_stage: 24개 (6.5%)
   - ajcc_pathologic_t: 2개 (0.5%)
   - ajcc_pathologic_n: 1개 (0.3%)
   - tumor_grade: 5개 (1.3%)
   - age_at_diagnosis: 3개 (0.8%)
   - treatments_pharmaceutical_treatment_intent_type: 44개 (11.8%)
   - treatments_pharmaceutical_treatment_type: 42개 (11.3%)
   - year_of_diagnosis: 2개 (0.5%)

🎯 3. 특성 준비 및 인코딩
🔄 CDSS 테스트용 환자 분리 중...
   - CDSS 테스트 환자: 332
   - 모델 훈련용

In [5]:
# 원본 데이터에서 holdout 환자 찾기
holdout_idx = treatment_predictor.processed_df.sample(n=1, random_state=42).index[0]

# 원본 데이터에서 해당 환자 정보 추출
if 'submitter_id' in treatment_predictor.df.columns:
    original_patient = treatment_predictor.df.loc[holdout_idx]
    print(f"🏥 Submitter ID: {original_patient['submitter_id']}")
    print(f"🆔 원본 인덱스: {holdout_idx}")
    
    # 추가 정보도 확인 가능
    print(f"👤 연령: {original_patient.get('age_at_diagnosis', 'N/A')}")
    print(f"👤 성별: {original_patient.get('gender', 'N/A')}")
    print(f"👤 생존 상태: {original_patient.get('vital_status', 'N/A')}")
else:
    print("⚠️ submitter_id 컬럼이 데이터에 없습니다")


🏥 Submitter ID: TCGA-CC-A9FS
🆔 원본 인덱스: 332
👤 연령: 20157.0
👤 성별: male
👤 생존 상태: Alive


In [6]:
# 원본 데이터에서 holdout 환자 찾기
holdout_idx = treatment_predictor.processed_df.sample(n=1, random_state=42).index[0]

# 출력할 모든 컬럼 리스트
detailed_columns = [
    "vital_status", "days_to_death", "days_to_last_follow_up", 
    "age_at_diagnosis", "gender", "race", "ethnicity",
    "ajcc_pathologic_stage", "ajcc_pathologic_t", "ajcc_pathologic_n", "ajcc_pathologic_m",
    "child_pugh_classification", "ishak_fibrosis_score", "tumor_grade",
    "primary_diagnosis", "tissue_or_organ_of_origin", "site_of_resection_or_biopsy",
    "morphology", "residual_disease", "classification_of_tumor",
    "prior_malignancy", "synchronous_malignancy", "prior_treatment",
    "treatments_pharmaceutical_treatment_type", "treatments_pharmaceutical_treatment_or_therapy",
    "treatments_pharmaceutical_treatment_intent_type", "treatments_radiation_treatment_type",
    "treatments_radiation_treatment_or_therapy", "treatments_radiation_treatment_intent_type",
    "year_of_diagnosis"
]

# 원본 데이터에서 해당 환자 정보 추출
if 'submitter_id' in treatment_predictor.df.columns:
    original_patient = treatment_predictor.df.loc[holdout_idx]
    
    print("="*60)
    print("🏥 HOLDOUT 환자 상세 정보")
    print("="*60)
    print(f"🆔 Submitter ID: {original_patient['submitter_id']}")
    print(f"🆔 원본 인덱스: {holdout_idx}")
    print("\n📋 상세 임상 정보:")
    print("-"*40)
    
    # 모든 상세 정보 출력
    for col in detailed_columns:
        if col in original_patient.index:
            value = original_patient[col]
            # None 값을 적절히 처리
            if pd.isna(value) or value is None:
                value = "None"
            print(f'        "{col}": "{value}",')
        else:
            print(f'        "{col}": "N/A",')
    
    print("\n📊 JSON 형태로 출력:")
    print("{")
    print(f'    "submitter_id": "{original_patient["submitter_id"]}",')
    print(f'    "index": {holdout_idx},')
    
    for col in detailed_columns:
        if col in original_patient.index:
            value = original_patient[col]
            if pd.isna(value) or value is None:
                print(f'    "{col}": null,')
            elif isinstance(value, str):
                print(f'    "{col}": "{value}",')
            else:
                print(f'    "{col}": {value},')
        else:
            print(f'    "{col}": "N/A",')
    print("}")
    
else:
    print("⚠️ submitter_id 컬럼이 데이터에 없습니다")


🏥 HOLDOUT 환자 상세 정보
🆔 Submitter ID: TCGA-CC-A9FS
🆔 원본 인덱스: 332

📋 상세 임상 정보:
----------------------------------------
        "vital_status": "Alive",
        "days_to_death": "None",
        "days_to_last_follow_up": "211.0",
        "age_at_diagnosis": "20157.0",
        "gender": "male",
        "race": "asian",
        "ethnicity": "not hispanic or latino",
        "ajcc_pathologic_stage": "Stage II",
        "ajcc_pathologic_t": "T2",
        "ajcc_pathologic_n": "N0",
        "ajcc_pathologic_m": "M0",
        "child_pugh_classification": "Unknown",
        "ishak_fibrosis_score": "None",
        "tumor_grade": "G2",
        "primary_diagnosis": "Hepatocellular carcinoma, NOS",
        "tissue_or_organ_of_origin": "Liver",
        "site_of_resection_or_biopsy": "Liver",
        "morphology": "8170/3",
        "residual_disease": "R0",
        "classification_of_tumor": "primary",
        "prior_malignancy": "no",
        "synchronous_malignancy": "No",
        "prior_treatment": "N